## 1. 데이터 로드 및 준비

각 모델별로 저장된 예측값과 실제값을 로드합니다.

In [ ]:
def load_model_results(model_name, data_path='../데이터/Results/'):
    """
    모델별 예측값과 실제값을 로드하는 함수
    
    Args:
        model_name (str): 모델 이름
        data_path (str): 데이터가 저장된 경로
    
    Returns:
        tuple: (predictions, targets) - 예측값과 실제값 리스트
    """
    try:
        # pickle 파일로 저장된 경우
        with open(f'{data_path}{model_name}_predictions.pkl', 'rb') as f:
            predictions = pickle.load(f)
        with open(f'{data_path}{model_name}_targets.pkl', 'rb') as f:
            targets = pickle.load(f)
    except:
        # numpy 파일로 저장된 경우
        try:
            predictions = np.load(f'{data_path}{model_name}_predictions.npy')
            targets = np.load(f'{data_path}{model_name}_targets.npy')
        except:
            print(f"Warning: {model_name} 데이터를 찾을 수 없습니다. 가상 데이터를 생성합니다.")
            # 가상 데이터 생성 (실제 사용시에는 이 부분을 제거하세요)
            np.random.seed(42)
            n_samples = 1000
            targets = np.random.normal(100, 20, n_samples)
            # 모델별로 다른 성능을 시뮬레이션
            noise_levels = {
                'lstm': 15, 'gru': 14, 'transformer': 12, 'informer': 13,
                'pyraformer': 11, 'n_beats': 16, 'nlinear': 18,
                'lstm_attention': 10, 'gru_attention': 9, 'transformer_attention': 8,
                'informer_attention': 9, 'pyraformer_attention': 7,
                'lightgbm': 14, 'catboost': 13
            }
            noise = noise_levels.get(model_name, 15)
            predictions = targets + np.random.normal(0, noise, n_samples)
    
    return np.array(predictions), np.array(targets)

In [ ]:
# 분석할 모델들 정의
models = [
    'lstm', 'gru', 'transformer', 'informer', 'pyraformer', 'n_beats', 'nlinear',
    'lstm_attention', 'gru_attention', 'transformer_attention', 'informer_attention', 'pyraformer_attention',
    'lightgbm', 'catboost'
]

# 각 모델별 데이터 로드
model_data = {}
model_errors = {}

for model in models:
    predictions, targets = load_model_results(model)
    model_data[model] = {'predictions': predictions, 'targets': targets}
    # 절대 오차 계산 (Wilcoxon 검정용)
    model_errors[model] = np.abs(predictions - targets)
    
    print(f"{model}: 샘플 수 = {len(predictions)}, MAE = {np.mean(model_errors[model]):.4f}")

## 2. Wilcoxon 부호-순위 검정 (Wilcoxon Signed-Rank Test)

두 모델 간의 성능 차이를 검정합니다. 같은 테스트 데이터에 대한 오차를 비교하여 통계적으로 유의한 차이가 있는지 확인합니다.

In [ ]:
def wilcoxon_pairwise_test(model_errors, alpha=0.05):
    """
    모든 모델 쌍에 대해 Wilcoxon 부호-순위 검정을 수행
    
    Args:
        model_errors (dict): 모델별 오차 딕셔너리
        alpha (float): 유의수준
    
    Returns:
        DataFrame: 검정 결과를 담은 데이터프레임
    """
    results = []
    model_names = list(model_errors.keys())
    
    for i, model1 in enumerate(model_names):
        for j, model2 in enumerate(model_names):
            if i < j:  # 중복 방지
                # Wilcoxon 부호-순위 검정
                statistic, p_value = wilcoxon(model_errors[model1], model_errors[model2])
                
                # 효과 크기 계산 (r = Z / sqrt(N))
                n = len(model_errors[model1])
                z_score = statistic / np.sqrt(n * (n + 1) / 6)
                effect_size = abs(z_score) / np.sqrt(n)
                
                # 결과 저장
                results.append({
                    'Model 1': model1,
                    'Model 2': model2,
                    'Statistic': statistic,
                    'P-value': p_value,
                    'Significant': 'Yes' if p_value < alpha else 'No',
                    'Effect Size': effect_size,
                    'Mean Error Diff': np.mean(model_errors[model1]) - np.mean(model_errors[model2])
                })
    
    return pd.DataFrame(results)

# Wilcoxon 검정 실행
wilcoxon_results = wilcoxon_pairwise_test(model_errors)

# 결과 정렬 (p-value 기준)
wilcoxon_results = wilcoxon_results.sort_values('P-value')

print("=== Wilcoxon 부호-순위 검정 결과 ===")
print(f"총 {len(wilcoxon_results)} 개의 모델 쌍 비교")
print(f"유의한 차이가 있는 쌍: {len(wilcoxon_results[wilcoxon_results['Significant'] == 'Yes'])} 개")
print("\n상위 10개 결과:")
print(wilcoxon_results.head(10).to_string(index=False))

In [ ]:
# Wilcoxon 검정 결과 시각화
plt.figure(figsize=(12, 8))

# P-value 히스토그램
plt.subplot(2, 2, 1)
plt.hist(wilcoxon_results['P-value'], bins=20, alpha=0.7, edgecolor='black')
plt.axvline(x=0.05, color='red', linestyle='--', label='α = 0.05')
plt.xlabel('P-value')
plt.ylabel('Frequency')
plt.title('P-value Distribution (Wilcoxon Test)')
plt.legend()

# 효과 크기 분포
plt.subplot(2, 2, 2)
plt.hist(wilcoxon_results['Effect Size'], bins=20, alpha=0.7, color='green', edgecolor='black')
plt.xlabel('Effect Size')
plt.ylabel('Frequency')
plt.title('Effect Size Distribution')

# 유의한 결과들의 효과 크기
significant_results = wilcoxon_results[wilcoxon_results['Significant'] == 'Yes']
plt.subplot(2, 2, 3)
if len(significant_results) > 0:
    plt.scatter(significant_results['P-value'], significant_results['Effect Size'], alpha=0.6)
    plt.xlabel('P-value')
    plt.ylabel('Effect Size')
    plt.title('Significant Results: P-value vs Effect Size')
else:
    plt.text(0.5, 0.5, 'No significant results', ha='center', va='center')
    plt.title('No Significant Results Found')

# 평균 오차 차이 분포
plt.subplot(2, 2, 4)
plt.hist(wilcoxon_results['Mean Error Diff'], bins=20, alpha=0.7, color='orange', edgecolor='black')
plt.axvline(x=0, color='red', linestyle='-', alpha=0.5)
plt.xlabel('Mean Error Difference')
plt.ylabel('Frequency')
plt.title('Mean Error Difference Distribution')

plt.tight_layout()
plt.show()

## 3. Friedman 검정 (Friedman Test)

여러 모델들 간의 전체적인 성능 차이를 검정합니다. 비모수 ANOVA의 대안으로, 모든 모델들이 동일한 성능을 가진다는 귀무가설을 검정합니다.

In [ ]:
def friedman_test(model_errors):
    """
    Friedman 검정을 수행하고 결과를 출력
    
    Args:
        model_errors (dict): 모델별 오차 딕셔너리
    
    Returns:
        tuple: (statistic, p_value, 검정 결과 딕셔너리)
    """
    # 모든 모델의 오차를 리스트로 변환
    error_lists = [model_errors[model] for model in models]
    
    # Friedman 검정 수행
    statistic, p_value = friedmanchisquare(*error_lists)
    
    # 검정 결과
    result = {
        'statistic': statistic,
        'p_value': p_value,
        'significant': p_value < 0.05,
        'df': len(models) - 1,
        'n_models': len(models),
        'n_samples': len(error_lists[0])
    }
    
    return statistic, p_value, result

# Friedman 검정 실행
friedman_stat, friedman_p, friedman_result = friedman_test(model_errors)

print("=== Friedman 검정 결과 ===")
print(f"검정 통계량: {friedman_stat:.4f}")
print(f"P-value: {friedman_p:.6f}")
print(f"자유도: {friedman_result['df']}")
print(f"모델 수: {friedman_result['n_models']}")
print(f"샘플 수: {friedman_result['n_samples']}")
print(f"유의성 (α=0.05): {'예' if friedman_result['significant'] else '아니오'}")

if friedman_result['significant']:
    print("\n✅ 모델들 간에 통계적으로 유의한 차이가 있습니다.")
    print("→ Nemenyi 사후 검정을 진행합니다.")
else:
    print("\n❌ 모델들 간에 통계적으로 유의한 차이가 없습니다.")
    print("→ 사후 검정이 필요하지 않습니다.")

In [ ]:
# 모델별 순위 계산 (낮은 오차가 더 좋은 순위)
def calculate_rankings(model_errors):
    """각 샘플에 대해 모델들의 순위를 계산"""
    n_samples = len(list(model_errors.values())[0])
    rankings = {model: [] for model in models}
    
    for i in range(n_samples):
        # i번째 샘플에서 각 모델의 오차
        sample_errors = [(model, model_errors[model][i]) for model in models]
        # 오차 기준으로 정렬 (낮은 오차가 좋은 순위)
        sample_errors.sort(key=lambda x: x[1])
        
        # 순위 할당
        for rank, (model, _) in enumerate(sample_errors, 1):
            rankings[model].append(rank)
    
    return rankings

# 순위 계산
rankings = calculate_rankings(model_errors)

# 평균 순위 계산
mean_rankings = {model: np.mean(rankings[model]) for model in models}
sorted_rankings = sorted(mean_rankings.items(), key=lambda x: x[1])

print("\n=== 모델별 평균 순위 (낮을수록 좋음) ===")
for i, (model, rank) in enumerate(sorted_rankings, 1):
    print(f"{i:2d}. {model:20s}: {rank:.3f}")

# 평균 순위 시각화
plt.figure(figsize=(12, 6))
models_sorted, ranks_sorted = zip(*sorted_rankings)
colors = plt.cm.RdYlGn_r(np.linspace(0.2, 0.8, len(models_sorted)))

bars = plt.bar(range(len(models_sorted)), ranks_sorted, color=colors, edgecolor='black', alpha=0.7)
plt.xlabel('Models')
plt.ylabel('Average Rank')
plt.title('Average Ranking of Models (Lower is Better)')
plt.xticks(range(len(models_sorted)), models_sorted, rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)

# 각 막대 위에 순위 값 표시
for i, (bar, rank) in enumerate(zip(bars, ranks_sorted)):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
             f'{rank:.2f}', ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.show()

## 4. Nemenyi 사후 검정 (Nemenyi Post-hoc Test)

Friedman 검정에서 유의한 차이가 발견된 경우, 어떤 모델들 간에 구체적으로 유의한 차이가 있는지 확인합니다.

In [ ]:
if friedman_result['significant']:
    # 데이터를 Nemenyi 검정에 맞는 형태로 변환
    # 각 행은 하나의 관측치(샘플), 각 열은 하나의 그룹(모델)
    n_samples = len(list(model_errors.values())[0])
    data_matrix = np.zeros((n_samples, len(models)))
    
    for i, model in enumerate(models):
        data_matrix[:, i] = model_errors[model]
    
    # DataFrame으로 변환
    df_nemenyi = pd.DataFrame(data_matrix, columns=models)
    
    print("=== Nemenyi 사후 검정 실행 중... ===")
    
    try:
        # Nemenyi 검정 수행
        nemenyi_results = sp.posthoc_nemenyi_friedman(df_nemenyi)
        
        print("\n=== Nemenyi 사후 검정 결과 ===")
        print("P-value 행렬 (α=0.05 기준):")
        print(nemenyi_results.round(4))
        
        # 유의한 차이가 있는 모델 쌍 찾기
        significant_pairs = []
        for i in range(len(models)):
            for j in range(i+1, len(models)):
                p_val = nemenyi_results.iloc[i, j]
                if p_val < 0.05:
                    model1, model2 = models[i], models[j]
                    rank1, rank2 = mean_rankings[model1], mean_rankings[model2]
                    significant_pairs.append({
                        'Model 1': model1,
                        'Model 2': model2,
                        'P-value': p_val,
                        'Rank 1': rank1,
                        'Rank 2': rank2,
                        'Rank Diff': abs(rank1 - rank2)
                    })
        
        if significant_pairs:
            print(f"\n✅ 총 {len(significant_pairs)}개의 모델 쌍에서 유의한 차이 발견:")
            sig_df = pd.DataFrame(significant_pairs)
            sig_df = sig_df.sort_values('P-value')
            print(sig_df.to_string(index=False))
        else:
            print("\n❌ 사후 검정에서 유의한 차이를 보이는 모델 쌍이 없습니다.")
            
    except Exception as e:
        print(f"Nemenyi 검정 중 오류 발생: {e}")
        print("대안으로 Bonferroni 보정된 Wilcoxon 검정을 수행합니다.")
        
        # Bonferroni 보정
        n_comparisons = len(wilcoxon_results)
        bonferroni_alpha = 0.05 / n_comparisons
        
        bonferroni_significant = wilcoxon_results[wilcoxon_results['P-value'] < bonferroni_alpha]
        
        print(f"\nBonferroni 보정 결과 (α = {bonferroni_alpha:.6f}):")
        if len(bonferroni_significant) > 0:
            print(f"유의한 차이가 있는 쌍: {len(bonferroni_significant)}개")
            print(bonferroni_significant[['Model 1', 'Model 2', 'P-value', 'Mean Error Diff']].to_string(index=False))
        else:
            print("Bonferroni 보정 후에도 유의한 차이를 보이는 쌍이 없습니다.")

else:
    print("\nFriedman 검정에서 유의한 차이가 없으므로 사후 검정을 수행하지 않습니다.")

In [ ]:
# Nemenyi 검정 결과 히트맵 시각화 (검정이 성공한 경우)
if friedman_result['significant']:
    try:
        plt.figure(figsize=(12, 10))
        
        # P-value 히트맵
        mask = np.triu(np.ones_like(nemenyi_results, dtype=bool))
        sns.heatmap(nemenyi_results, mask=mask, annot=True, cmap='RdYlBu_r', 
                   center=0.05, square=True, fmt='.3f', cbar_kws={"shrink": .8})
        plt.title('Nemenyi Post-hoc Test Results\n(P-values, α=0.05)')
        plt.ylabel('Models')
        plt.xlabel('Models')
        plt.xticks(rotation=45, ha='right')
        plt.yticks(rotation=0)
        plt.tight_layout()
        plt.show()
        
        # 유의성 히트맵 (유의한 차이가 있으면 1, 없으면 0)
        significance_matrix = (nemenyi_results < 0.05).astype(int)
        
        plt.figure(figsize=(12, 10))
        sns.heatmap(significance_matrix, mask=mask, annot=True, cmap='RdBu_r',
                   center=0.5, square=True, fmt='d', cbar_kws={"shrink": .8})
        plt.title('Statistical Significance Matrix\n(1: Significant difference, 0: No significant difference)')
        plt.ylabel('Models')
        plt.xlabel('Models')
        plt.xticks(rotation=45, ha='right')
        plt.yticks(rotation=0)
        plt.tight_layout()
        plt.show()
        
    except:
        print("히트맵 생성 중 오류가 발생했습니다.")

## 5. 결과 요약 및 결론

In [ ]:
print("="*60)
print("                     최종 결과 요약")
print("="*60)

# 1. 기본 통계
print("\n1. 기본 통계:")
print(f"   - 분석 모델 수: {len(models)}개")
print(f"   - 테스트 샘플 수: {len(list(model_errors.values())[0])}개")
print(f"   - 모델 쌍 비교 수: {len(wilcoxon_results)}개")

# 2. Wilcoxon 검정 결과
n_significant_wilcoxon = len(wilcoxon_results[wilcoxon_results['Significant'] == 'Yes'])
print(f"\n2. Wilcoxon 부호-순위 검정:")
print(f"   - 유의한 차이가 있는 모델 쌍: {n_significant_wilcoxon}개 / {len(wilcoxon_results)}개")
print(f"   - 유의성 비율: {n_significant_wilcoxon/len(wilcoxon_results)*100:.1f}%")

# 3. Friedman 검정 결과
print(f"\n3. Friedman 검정:")
print(f"   - 검정 통계량: {friedman_stat:.4f}")
print(f"   - P-value: {friedman_p:.6f}")
print(f"   - 전체적인 모델 간 차이: {'있음' if friedman_result['significant'] else '없음'}")

# 4. 모델 순위
print(f"\n4. 모델 성능 순위 (평균 순위 기준):")
for i, (model, rank) in enumerate(sorted_rankings[:5], 1):
    print(f"   {i}위: {model} (평균 순위: {rank:.3f})")
if len(sorted_rankings) > 5:
    print(f"   ... (총 {len(sorted_rankings)}개 모델)")

# 5. 사후 검정 결과
if friedman_result['significant']:
    try:
        if 'significant_pairs' in locals() and significant_pairs:
            print(f"\n5. Nemenyi 사후 검정:")
            print(f"   - 유의한 차이가 있는 모델 쌍: {len(significant_pairs)}개")
            print(f"   - 가장 큰 성능 차이: {max([p['Rank Diff'] for p in significant_pairs]):.3f}")
        else:
            print(f"\n5. 사후 검정 결과: 유의한 차이를 보이는 모델 쌍 없음")
    except:
        print(f"\n5. 사후 검정: 오류로 인해 완료되지 않음")
else:
    print(f"\n5. 사후 검정: Friedman 검정에서 유의한 차이가 없어 수행하지 않음")

print("\n" + "="*60)
print("분석 완료!")
print("="*60)

In [ ]:
import numpy as np
import pandas as pd
import pickle
from scipy import stats
from scipy.stats import friedmanchisquare, wilcoxon
import scikit_posthocs as sp
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
import warnings
warnings.filterwarnings('ignore')

# 통계 검정 분석

이 노트북에서는 다양한 모델들의 성능을 비교하기 위해 다음 통계 검정들을 수행합니다:

1. **Wilcoxon 부호-순위 검정 (Wilcoxon Signed-Rank Test)**
   - 두 모델 간의 성능 차이를 검정
   - 쌍체 비모수 검정

2. **Friedman 검정 (Friedman Test)**
   - 여러 모델들 간의 전체적인 성능 차이를 검정
   - 비모수 ANOVA의 대안

3. **Nemenyi 사후 검정 (Nemenyi Post-hoc Test)**
   - Friedman 검정에서 유의한 차이가 발견된 경우
   - 어떤 모델들 간에 유의한 차이가 있는지 확인